In [1]:
# %%capture
# %%script echo skipping
%load_ext autoreload
%load_ext dotenv
%dotenv
%pip install transformers evaluate
%pip install nltk absl-py rouge_score
%pip install bleu sacrebleu
%pip install sacremoses
%pip install scipy
%pip install sentencepiece
%pip install optimum auto-gptq
%pip install scikit-learn
%pip install einops
%pip install bitsandbytes
%pip install accelerate
%pip install pynvml
!huggingface-cli login --token $HUGGING_FACE_TOKEN


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/papers

In [2]:
import os
import sys
from pprint import pprint

import evaluate
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from huggingface_hub import HfApi
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BartForCausalLM,
    BartModel,
    BartTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    T5ForConditionalGeneration,
    T5Tokenizer,
    T5TokenizerFast,
)

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from utils.metric import calculate_scores
from utils.monitoring import calculate_utilization, format_utilization_narrow, print_utilization

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pprint(f"Device: {device}")

/home/paperspace/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'Device: cuda'


In [3]:
%reload_ext autoreload

utilization = calculate_utilization()
utilization_str = format_utilization_narrow(utilization)
print(
    f"total/used/cuda/res/ram (Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
    f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']}"
)


available_memory = utilization["total_memory"] - utilization["memory_used"]
recommended_fraction = available_memory / utilization["total_memory"]

actual_fraction = 0.95
torch.cuda.set_per_process_memory_fraction(actual_fraction, 0)

print(
    f"Total/used/available memory (Gb): {utilization['total_memory']/1024**3:.2f}/{utilization['memory_used']/1024**3:.2f}/{available_memory/1024**3:.2f}"
    f"Recommended/actual fraction: {recommended_fraction:.2f}/{actual_fraction:.2f}"
)

# torch.cuda.empty_cache()
# torch.empty(utilization['total_memory'] // 2, dtype=torch.int8, device='cuda')
# print_utilization()

total/used/cuda/res/ram (Gb): 79.33/0.84/0.00/0.00/3.37
Total/used/available memory (Gb): 79.33/0.84/78.49Recommended/actual fraction: 0.99/0.95


## Loading models

### Training BART (SequenceToSequence) on grammarly-coedit dataset
* https://huggingface.co/facebook/bart-large

In [4]:
checkpoint = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(checkpoint)

print(type(tokenizer))
# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state


<class 'transformers.models.bart.tokenization_bart.BartTokenizer'>


In [5]:
# api = HfApi()
# coedit_info = api.dataset_info("grammarly/coedit")
# pprint(coedit_info)

# full_data = load_dataset("grammarly/coedit")

data = load_dataset("grammarly/coedit", split="train[:1000]")
# data = data.train_test_split(test_size=0.2)

data = load_dataset("grammarly/coedit")
data["test"] = data["validation"]
del data["validation"]

print(data)
print(data["train"])
print(data["train"][0]['src'])
print(data["test"])
print(data["test"][0]['src'])

# unique_categories = set(grammarly_dataset)
# pprint(unique_categories)

# unique_tasks = set(grammarly_dataset["train"]["task"])
# pprint(unique_tasks)

DatasetDict({
    train: Dataset({
        features: ['_id', 'task', 'src', 'tgt'],
        num_rows: 69071
    })
    test: Dataset({
        features: ['_id', 'task', 'src', 'tgt'],
        num_rows: 1712
    })
})
Dataset({
    features: ['_id', 'task', 'src', 'tgt'],
    num_rows: 69071
})
Remove all grammatical errors from this text: For example, countries with a lot of deserts can terraform their desert to increase their habitable land and using irrigation to provide clean water to the desert.
Dataset({
    features: ['_id', 'task', 'src', 'tgt'],
    num_rows: 1712
})
Paraphrase this sentence: Why are you arresting me?


In [6]:
# test tokenizer

print(data)
sample = data["train"][0]['src']
samples = [sample, sample]
input_ids = tokenizer(samples, is_split_into_words=True, padding=True).input_ids
print(input_ids)
# result = tokenizer.convert_ids_to_tokens(input_ids)
# print(result)
decoded = tokenizer.decode(input_ids, skip_special_tokens=True)
print(decoded)

DatasetDict({
    train: Dataset({
        features: ['_id', 'task', 'src', 'tgt'],
        num_rows: 69071
    })
    test: Dataset({
        features: ['_id', 'task', 'src', 'tgt'],
        num_rows: 1712
    })
})
[0, 27336, 70, 25187, 45816, 9126, 31, 42, 2788, 35, 286, 1246, 6, 749, 19, 10, 319, 9, 38905, 64, 8470, 763, 3899, 49, 10348, 7, 712, 49, 40752, 1212, 8, 634, 20500, 7, 694, 2382, 514, 7, 5, 10348, 4, 27336, 70, 25187, 45816, 9126, 31, 42, 2788, 35, 286, 1246, 6, 749, 19, 10, 319, 9, 38905, 64, 8470, 763, 3899, 49, 10348, 7, 712, 49, 40752, 1212, 8, 634, 20500, 7, 694, 2382, 514, 7, 5, 10348, 4, 2]
 Remove all grammatical errors from this text: For example, countries with a lot of deserts can terraform their desert to increase their habitable land and using irrigation to provide clean water to the desert. Remove all grammatical errors from this text: For example, countries with a lot of deserts can terraform their desert to increase their habitable land and using irrigati

In [7]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["src"], max_length=1024, padding=True)
    labels = tokenizer(text_target=examples["tgt"], max_length=1024, padding=True).input_ids
    model_inputs["labels"] = labels
    return model_inputs


# processed_data = data.map(preprocess_function, batched=True, batch_size=10, remove_columns=data["train"].column_names)
processed_data = data.map(preprocess_function, batched=True, batch_size=10, remove_columns=data["train"].column_names)
print(processed_data)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 69071
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1712
    })
})


In [8]:
train_data = processed_data['train']
print(train_data[0])
print(len(train_data[0]['input_ids']))
print(train_data[0])
print(len(train_data[0]['attention_mask']))
print(len(train_data[0]['labels']))
# print(train_data[8])
# print(len(train_data[8]['input_ids']))
# print(train_data[9])
# print(len(train_data[9]['input_ids']))
# print(train_data[10])
# print(len(train_data[10]['input_ids']))
# print(train_data[11])
# print(len(train_data[11]['input_ids']))

{'input_ids': [0, 47583, 70, 25187, 45816, 9126, 31, 42, 2788, 35, 286, 1246, 6, 749, 19, 10, 319, 9, 38905, 64, 8470, 763, 3899, 49, 10348, 7, 712, 49, 40752, 1212, 8, 634, 20500, 7, 694, 2382, 514, 7, 5, 10348, 4, 2, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 'labels': [0, 2709, 1246, 6, 749, 19, 10, 319, 9, 38905, 64, 7891, 49, 10348, 7, 712, 49, 40752, 1212, 8, 304, 20500, 7, 694, 2382, 514, 7, 5, 10348, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
49
{'input_ids': [0, 47583, 70, 25187, 45816, 9126, 31, 42, 2788, 35, 286, 1246, 6, 749, 19, 10, 319, 9, 38905, 64, 8470, 763, 3899, 49, 10348, 7, 712, 49, 40752, 1212, 8, 634, 20500, 7, 694, 2382, 514, 7, 5, 10348, 4, 2, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0

In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map=0)
print(type(model))
print(model.config)

total_params = sum(p.numel() for p in model.parameters())
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total/trainable params: {total_params}/{total_trainable_params}')

utilization = calculate_utilization()
utilization_str = format_utilization_narrow(utilization)
print(
    f"total/used/cuda/res/ram (Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
    f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']}"
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)
rouge = evaluate.load("rouge")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

<class 'transformers.models.bart.modeling_bart.BartForConditionalGeneration'>
BartConfig {
  "_name_or_path": "facebook/bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    

In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir="bart-grammarly-train",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=150,
    per_device_eval_batch_size=150,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    # push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_data["train"],
    eval_dataset=processed_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("bart-grammarly-model")

/home/paperspace/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.246618,0.607700,0.481000,0.575300,0.575800,17.615700
2,1.292400,0.223714,0.612200,0.493200,0.585500,0.586200,17.916500
3,0.421400,0.221330,0.613600,0.494800,0.590000,0.590500,17.745900
4,0.381000,0.222419,0.615000,0.496400,0.590500,0.591300,17.702700


/home/paperspace/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/home/paperspace/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_

KeyboardInterrupt: 

In [19]:
!huggingface-cli login --token $HUGGING_FACE_TOKEN

trainer.push_to_hub("bart-grammarly")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/paperspace/.cache/huggingface/token
Login successful


RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-661c28e9-22126e8578a845493cd62efd;3dc8e6ad-f352-4fb1-a03c-4ca368e1329e)

Repository Not Found for url: https://huggingface.co/api/models/bart-grammarly-model.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [25]:
from transformers import BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('./bart-grammarly-model')
model = BartForConditionalGeneration.from_pretrained('./bart-grammarly-model', device_map=0)
# print(model.config)

model.push_to_hub("iliazlobin/bart-grammarly", token=os.getenv("HUGGING_FACE_TOKEN"))

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
model.safetensors:  58%|█████▊    | 944M/1.63G [00:15<00:09, 68.8MB/s] 

In [6]:
from transformers import BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('./bart-grammarly-model')
model = BartForConditionalGeneration.from_pretrained('./bart-grammarly-model', device_map=0)
# print(model.config)

# input_text = "Remove all grammatical errors from this text: For example, countries with a lot of deserts can terraform their desert to increase their habitable land and using irrigation to provide clean water to the desert."
input_text = "Fix grammar: I plays football."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = model.generate(**input_ids, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

output = "For example, countries with a lot of deserts can terraform their desert to increase their habitable land and use irrigation to provide clean water to the desert"
reference = "For example, countries with a lot of deserts can transform their desert to increase their habitable land and use irrigation to provide clean water to the desert."

I play football.
